In [ ]:
import numpy as np
import pandas as pd
# Loading the data into the environment using pandas
# Note: Please use appropriate filename and path
sms_data = pd.read_csv("spam.csv", encoding='latin-1')
# Review the loaded data
print(sms_data.head())
cols = sms_data.columns[:2]
data = sms_data[cols]
print(data.shape)
data = data.rename(columns={"v1":"Value","v2":"Text"})
print(data.head())
print(data.Value.value_counts())


Feature Engineering

In [5]:
from string import punctuation
import re
import nltk
from nltk import word_tokenize



In [6]:
punctuation = list(punctuation)
# Creating a new feature called Punctuations. 
# This feature counts the number of punctuation characters in the sms message 
data["Punctuations"] = data["Text"].apply(lambda x: len(re.findall(r"[^\w+&&^\s]",x)))
# Creating a new feature called Phonenumbers. 
# This feature indicates if the sms text contains a phonenumber or not
data["Phonenumbers"] = data["Text"].apply(lambda x: len(re.findall(r"[0-9]{10}",x)))


In [7]:
# Creating a new feature called Links.
# This feature indicates if the sms text contains a URL or not 
is_link = lambda x: 1 if re.search(r"https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+",x)!=None else 0
data["Links"] = data["Text"].apply(is_link)
# Creating a new feature called Uppercase.
# This feature indicates how many words in the the sms text are in upper case
count_upper = lambda x : list(map(str.isupper,x.split())).count(True)
upper_case = lambda y,n : n+1 if y.isupper() else n
data["Uppercase"] = data["Text"].apply(count_upper)


In [8]:
print(data.head(2))

  Value                                               Text  Punctuations  \
0   ham  Go until jurong point, crazy.. Available only ...             9   
1   ham                      Ok lar... Joking wif u oni...             6   

   Phonenumbers  Links  Uppercase  
0             0      0          0  
1             0      0          0  


In [ ]:
# # Identifying and counting how many unusual words are there in the sms text
# def find_unusual_words(text):
#     text_vocab_set = set(w.lower() for w in text if w.isalpha())
#     english_vocab_set = set(w.lower() for w in nltk.corpus.words.words())
#     unusual_set = text_vocab_set - english_vocab_set
#     return len(sorted(unusual_set))
#data["unusualwords"] = data["Text"].apply(lambda x: find_unusual_words(word_tokenize(x)))
# View a few records of the data after creating these features
#print(data[14:25])

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf= TfidfVectorizer(stop_words="english",strip_accents='ascii',max_features=300)
tf_idf_matrix = tf_idf.fit_transform(data["Text"])


In [13]:
data_extra_features = pd.concat([data,pd.DataFrame(tf_idf_matrix.toarray(),columns=tf_idf.get_feature_names_out())],axis=1)


In [15]:
print(data_extra_features.head(2))

  Value                                               Text  Punctuations  \
0   ham  Go until jurong point, crazy.. Available only ...             9   
1   ham                      Ok lar... Joking wif u oni...             6   

   Phonenumbers  Links  Uppercase  000   10  150p  150ppm  ...     world  www  \
0             0      0          0  0.0  0.0   0.0     0.0  ...  0.594379  0.0   
1             0      0          0  0.0  0.0   0.0     0.0  ...  0.000000  0.0   

   xmas  xxx   ya  yeah  year  yes   yo  yup  
0   0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  
1   0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  

[2 rows x 306 columns]


In [16]:
from sklearn.model_selection import train_test_split
X=data_extra_features
features = X.columns.drop(["Value","Text"])
target = ["Value"]
X_train,X_test,y_train,y_test = train_test_split(X[features],X[target])


In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
dt = DecisionTreeClassifier(min_samples_split=40)
dt.fit(X_train,y_train)
pred = dt.predict(X_test)
print(accuracy_score(y_train, dt.predict(X_train)))
print(accuracy_score(y_test, pred))


0.9827709978463748
0.9734386216798278


In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Building a Naive Bayes Model
mnb = MultinomialNB()
mnb.fit(X_train,y_train)
pred_mnb = mnb.predict(X_test)
print(accuracy_score(y_test, pred_mnb))
# Building a Logistic Regression Model
lr = LogisticRegression()
lr.fit(X_train,y_train)
pred_lr = lr.predict(X_test)
print(accuracy_score(y_test, pred_lr))


c:\Users\91827\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\91827\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9748743718592965
0.9777458722182341


c:\Users\91827\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
